# Data Wrangling Exercise
Purpose: Classify learners by CEFR

Phase 1: Wrangle some datums

Notes from meeting with Scott:
1. Consider text length
2. Consider how representative each text is (e.g. of a given CEFR band). I am not sure if he was alluding to outliers or something else here.
3. Methods/Technologies to consider:

    a. Semantic spaces

    b. LSA (this was a strong suggestion)

    c. Word2Vec

## Information on EFCAMDAT

> EFCAMDAT consists of essays submitted to Englishtown, the online school of EF Education First, by language learners all over the world (Education First, 2012).  A full course in Englishtown spans 16 proficiency levels aligned with common standards such as TOEFL, IELTS and the Common European Framework of Reference for languages.

__[Overview of EFCAMDAT Data (2013)](https://corpus.mml.cam.ac.uk/faq/SLRF2013Geertzenetal.pdf)__

__[Study with recommendations for Dependency Parsing on this data set (2018)](https://corpus.mml.cam.ac.uk/faq/IJCL2018Huangetal.pdf)__



In [123]:
from lxml import etree
import re # one-way encryption for your codebase
import os.path
import unicodedata # manipulate strange unicode characters
from IPython.display import display # Show me what's going on.
import pandas as pd

print('Working Directory set to:', os.getcwd())

# test_file = os.path.join(os.pardir, 'Original Files', 'Level 6 EF_camdat.txt')
# with open(test_file, "r") as file:
#     data = file.read()

Working Directory set to: /home/jovyan/work/efcamdat/efcamdat-data-cleaning


### Convert the text to XML
One option is to manually alter each illegal character into well-formatted XML. A full-featured text editor like Notepad++ might be a good fit for the job.
Another option is to wrap every `<TEXT>` block in `<![CDATA[]]>` tags, which might magically make the XML properly formatted.

__[Predefined characters in XML](https://en.wikipedia.org/wiki/List_of_XML_and_HTML_character_entity_references#Predefined_entities_in_XML)__ are mainly `& "  '  <  and >`

__[CDATA Sections in XML](https://www.tutorialspoint.com/xml/xml_cdata_sections.htm)__

In [124]:
# I like getting rid of excess whitespace, but let's temporarily remove that feature so we can examine the Diff more clearly.
# def wrap_cdata(text_masquerading_as_xml):
#     return re.sub(r'\s*</text>',r']]></text>', re.sub(r'<text>\s*',r'<text><![CDATA[', text_masquerading_as_xml))

def wrap_cdata(text_masquerading_as_xml):
    return re.sub(r'</text>',r']]></text>', re.sub(r'<text>',r'<text><![CDATA[', text_masquerading_as_xml))

# cdata_blocked_off = wrap_cdata(data)

### This seems to work for some of the levels, but not all of them. Let's try to troubleshoot some more.
Okay. It seems all(?) of the remaining issues are control characters. Python package unicodedata can handle this (thank you
StackOverflow). One way to double check that this is working as expected would be to look at a git difference between
cdata_blocked_off and controls_removed. This way we can ensure that only control characters are being removed.
At first I thought Scott might have intentionally made this data difficult to work with, but this is way
too crazy for anyone to have done it on purpose. One of the responses is literally just a string of control characters??

The remove_control_characters function could be narrowed in scope to just the text within CDATA tags, but I am pretty sure there are no problem bytes anywhere else... therefore it would have the same effect.

In [132]:
def remove_control_characters(not_really_xml):
    return "".join(ch for ch in not_really_xml if unicodedata.category(ch)[0]!="C")

# test_file = os.path.join(os.pardir, 'Original Files', 'Level 3 EF_camdat.txt')
# with open(test_file, "r") as file:
#     raw_test = file.read()
#     clean_test = remove_control_characters(wrap_cdata(raw_test))
    
# for c in clean_test:
#     print(i, '%04x' % ord(c), unicodedata.category(c), end=" ")
    
    
# controls_removed = remove_control_characters(cdata_blocked_off)

Ok, there are some discrepancies between Shatz's report and the data I am seeing. Let's actually look at the git diffs to see if I have made a mistake. This will be a good tool to acquire anyway.

In [126]:
import difflib
from pprint import pprint
test_file = os.path.join(os.pardir, 'Original Files', 'Level 16 EF_camdat.txt')
with open(test_file, "r") as file:
    raw_test = file.read()
    clean_test = remove_control_characters(wrap_cdata(raw_test))

d = difflib.Differ()
text1 = raw_test.splitlines(keepends=True)
text2 = clean_test.splitlines(keepends=True)

result = list(d.compare(text1,text2))
for line in result[-100:]:
    if line[0] in ['-','+','?']:
        pprint(line)

'-             <learner id="18538337" nationality="br"/>\n'
'-             <topic id="9186">Researching a legendary creature</topic>\n'
'-             <date>2012-08-06 19:33:11.493</date>\n'
'-             <grade>89</grade>\n'
'-             <text>\n'
('-             Legend: Chupacabra  Description: Animal that sucks the blood '
 'of farm animals, especially goats.  Background information: Attack and '
 'drinks blood of farm animals.  Physical features: Size of small bear; row of '
 'spines from neck to tail; mixture of a dog, rat and kangaroo; fangs; '
 'hairless; reptile-like with scales; stands 3-4 feet and hops like a '
 'kangaroo; forked tongue; hisses and screeches and eyes glow red; has a bad '
 'stench.  First reported: in Puerto Rico in 1995. Eight shep were attacked '
 'and blood sucked out. Only puncture marks are found on animal. Months after '
 'his report, 150 farm animals have blood drained and small puncture wounds.  '
 'Sightings: Puerto Rico, Mexico, Latin America, Un

In [127]:
def get_writings(cleanish_xml):
    return etree.fromstring(bytes(cleanish_xml, encoding='utf8'))[1]

# parsed_xml_writings = get_writings(controls_removed)
# print("The number of \'samples\' in this level {:,}:".format(len(parsed_xml_writings)))
#some of these samples are obviously useless...

### This is looking good! Now we need to build a dataframe and extract the texts.

In [128]:
# Let's just hard code this...

def xml_framer(xml_root, cols):
    lol = [] # list of lists
    for sample in xml_root:
        l = []
        l.append(sample.attrib['id'])
        l.append(sample.attrib['level'])
        l.append(sample.attrib['unit'])
        l.append(sample[0].attrib['id'])
        l.append(sample[0].attrib['nationality'])
        l.append(sample[1].text)
        l.append(sample[1].attrib['id'])
        l.append(sample[2].text)
        l.append(sample[3].text)
        l.append(sample[4].text)
        lol.append(l)
    df = pd.DataFrame(lol, columns=cols)
    return df

col_labels = ['id','lvl','unit','author_id','author_nationality','topic','topic_id','date','grade','text']

# test_df = xml_framer(parsed_xml_writings, col_labels)
# display(test_df)

### Putting it all together
If you have the RAM, might as well...

**Manually fix the mismatched tags (`<user>Theodora Alexopoulou</url>`) in line 8 of Level 6 EF_camdat.txt**

In [134]:
df = pd.DataFrame(columns=col_labels)
for i in range(1,17):
    file = os.path.join(os.pardir, 'Original Files', ' '.join(['Level', str(i), 'EF_camdat.txt']))
    with open(file, "r") as f:
        data = f.read()
        df = df.append(xml_framer(get_writings(remove_control_characters(wrap_cdata(data))), col_labels)).reset_index(drop=True)
display(df)

,id,lvl,unit,author_id,author_nationality,topic,topic_id,date,grade,text
0,C18217,1,7,20967052,cn,Writing labels for a clothing store,22440,2012-04-20 21:12:14.890,79,Date:monday 11th. Time:9.30 am. ...
1,C18541,1,1,21016113,cn,Introducing yourself by email,3535,2011-12-24 03:50:49.100,85,"Dear teacher, My name's Yi Zhao,En..."
2,C18648,1,1,20967075,cn,Introducing yourself by email,3535,2012-04-20 08:53:55.087,90,"My name's Henry Hong, I was born ..."
3,C20184,1,1,18898793,cn,Introducing yourself by email,3535,2011-12-14 08:54:48.380,95,"Dear Sir, I'm Jianwen Zhang, from ..."
4,C20185,1,2,18898793,cn,Taking inventory in the office,9820,2011-12-26 09:49:48.140,98,"Dear Ms Thomas, There are thirteen..."
...,...,...,...,...,...,...,...,...,...,...
549276,U718945,16,2,21649452,us,Writing about a symbol of your country,8341,2012-09-04 02:47:23.180,65,THE SPACE NEEDLE The United States...
549277,U719493,16,1,24750663,ae,Attending a robotics conference,7524,2012-09-05 20:41:18.933,0,This is a most new matter of proto...
549278,U722572,16,5,22117882,br,Using creative writing techniques,9146,2012-09-17 20:59:34.413,96,"Hi Tabby, Never mind about askin..."
549279,U724086,16,2,19328467,sa,Writing about a symbol of your country,8341,2012-09-23 06:55:59.733,94,Oil Tower Statue It is a statue ...


In [157]:
display(len(set(df['topic_id'])))

display(df['topic_id'].value_counts()[df['topic_id'].value_counts() > 10])
display(df['topic_id'].value_counts()[df['topic_id'].value_counts() > 10])
# display(df[df['text'].str.contains('</br>', regex=False)])

243

3535     42058
9820     25742
8967     22243
5322     21124
8572     17395
         ...  
28243       47
23235       45
28282       39
225         21
28257       13
Name: topic_id, Length: 156, dtype: int64

In [31]:
df.to_csv('all_levels.csv') # about 200MB

### We may want to consider running a spellchecker on these responses.

Maybe we...

1. Eliminate useless responses. I think we can assume that all nearly-identical responses have language taken from the prompt. Even if these responses are not just echos of the prompt, I do not think they can tell us very much about the writer, since there is very little variation between them. Like most things, I'm not sure about this.

    a. I think it could be fun to calculate levenshtein distance on the responses, and eliminate responses that are too similar that way.
    
    b. I think the more robust method would be to calculate the TF-IDF and cosine similarity, but that seems a little complex to just find responses with low variance.
    
    c. We could use a cutoff with the "Grade attribute". Irrelevant or incomplete responses are graded lower (maybe anything below 50? below 70?).
    
2. Once we have a relatively useful subset of samples, we can bifurcate

    a. Create .spacy on the samples
    
    b. Run spellchecker then create .spacy on a copy of the samples. This might not buy us anything, but it just might make a difference.
    
    **Actually, I think the best way to do this would be through spaCy.** I don't know if spaCy has an in-built spell checker, but we could add a spell checker to a custom spaCy pipeline. This would have the important advantage of preserving the original, misspelled response as well as a reasonable prediction of the intended orthography. I am not sure how we could incorporate both the uncorrected and corrected texts into a single model, but I sort of like the workflow here anyway.

### Errors I've noticed
1. Spaces before commas. I think this is **always** an error.
2. No space after commas. I think this is an error unless the character after the comma is a quotation mark.
3. Spaces before apostrophes/inverted commas. I think this is **always** an error.
4. Misspellings. I suspect we can improve the data automatically by spell checking, but we of course lose information about writers' spelling knowledge.
5. No space after periods (at end of sentence). This one is tricky. Periods should have a space when they are separating a sentence, but not when they are separating numbers (69.00 dollars) or acronyms (U.S.A.). 

spaCy handles most of these errors pretty well I think. It's probably not worth worrying about them. Even where I have marked the error as highly predictable (**always**), a lot of testing would be necessary.